# Constraint results inspector

In [12]:
import pyoxigraph
from pyoxigraph import RdfFormat
import pandas as pd

In [13]:
store = pyoxigraph.Store()

In [14]:
store.load(open("./results.nq", 'rb'), RdfFormat.N_QUADS, base_iri=None, to_graph=None)

```
SELECT ?id ?name ?severity ?grname (STR(?refurl) AS ?str_refurl) (STR(?refresource) AS ?str_refresrouce) ?constraint ?path ?message (STR(?focus) AS ?focusURL) ?focusType ?value
```

```
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/ns/shacl#focusNode
http://www.w3.org/ns/shacl#resultMessage
http://www.w3.org/ns/shacl#resultPath
http://www.w3.org/ns/shacl#resultSeverity
http://www.w3.org/ns/shacl#sourceConstraintComponent
http://www.w3.org/ns/shacl#sourceShape
```


In [15]:
sparql = """PREFIX shacl: <http://www.w3.org/ns/shacl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>

SELECT ?id ?name ?severity ?grname (STR(?refurl) AS ?str_refurl) (STR(?refresource) AS ?str_refresrouce) ?constraint ?path ?message (STR(?focus) AS ?focusURL) ?focusType ?value
WHERE {
  ?id rdf:type shacl:ValidationResult .
  OPTIONAL { ?id shacl:sourceShape ?ss . }
  OPTIONAL {
     ?id shacl:focusNode ?focus .
    ?focus rdf:type ?focusType .
  }
  OPTIONAL { ?id shacl:resultMessage ?message . }
  OPTIONAL { ?id shacl:resultSeverity ?severity . }
  OPTIONAL { ?id shacl:sourceConstraintComponent ?constraint . }
  OPTIONAL {  ?ss shacl:name ?name . }
  OPTIONAL {
    ?ss shacl:group ?group .
    ?group rdfs:label ?grname .
    ?group schema:url ?refurl .
    ?group rdfs:isDefinedBy ?refresource .
   }
  OPTIONAL { ?id shacl:resultPath ?path . }
  OPTIONAL { ?id shacl:value ?value . }
}
"""

r = store.query(sparql)
q1 = list(r)

In [16]:
print(list(r))

[]


In [17]:
# get the list of the variables from the query for ues in the dataframe
vars = r.variables
value_list = [variable.value for variable in vars]

In [18]:
# This map is necessary to get the values from the pyoxigraph.QuerySolutions
def extract_value(cell):
    if isinstance(cell, (pyoxigraph.Literal, pyoxigraph.NamedNode, pyoxigraph.BlankNode)):
        return cell.value
    return cell


In [19]:
df = pd.DataFrame(q1, columns=value_list)


In [20]:
df = df.applymap(extract_value)

/tmp/ipykernel_1356142/1674585045.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(extract_value)


In [21]:
df


,id,name,severity,grname,str_refurl,str_refresrouce,constraint,path,message,focusURL,focusType,value
0,c10cb2867f720243aed5c36d84e00295,None,http://www.w3.org/ns/shacl#Violation,None,None,None,http://www.w3.org/ns/shacl#QualifiedMinCountCo...,https://schema.org/variableMeasured,At least one PropertyValue in variableMeasured...,None,None,None
1,d30fd099d52d209d3263422ec0a591e0,None,http://www.w3.org/ns/shacl#Warning,None,None,None,http://www.w3.org/ns/shacl#MinCountConstraintC...,https://schema.org/variableMeasured,variable measured check,None,None,None
2,ced81c4c9a9cb3617e1c63f9ed7e030a,None,http://www.w3.org/ns/shacl#Warning,None,None,None,http://www.w3.org/ns/shacl#MinCountConstraintC...,https://schema.org/measurementMethod,measurement method check,None,None,None
3,f4d892e7e79f27eda7e661a077464e0b,None,http://www.w3.org/ns/shacl#Warning,None,None,None,http://www.w3.org/ns/shacl#MinCountConstraintC...,https://schema.org/contacts,Contact information should be provided,None,None,None
4,f211a050478c1dab075a737e3e6b07a9,None,http://www.w3.org/ns/shacl#Violation,None,None,None,http://www.w3.org/ns/shacl#MinCountConstraintC...,https://schema.org/license,License information should be provided,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
6995,c203e466c0139893095898708a29edab,None,http://www.w3.org/ns/shacl#Violation,None,None,None,http://www.w3.org/ns/shacl#QualifiedMinCountCo...,https://schema.org/variableMeasured,At least one PropertyValue in variableMeasured...,None,None,None
6996,ed8e5157a41af87d7280174d4f7358f7,None,http://www.w3.org/ns/shacl#Violation,None,None,None,http://www.w3.org/ns/shacl#QualifiedMinCountCo...,https://schema.org/variableMeasured,At least one PropertyValue in variableMeasured...,None,None,None
6997,c1c3c2f75a99bc7f9fd8fbc8199d2e52,None,http://www.w3.org/ns/shacl#Warning,None,None,None,http://www.w3.org/ns/shacl#MinCountConstraintC...,https://schema.org/variableMeasured,variable measured check,None,None,None
6998,b45bdd94ea1c29fdeed7e5b9dea7660e,None,http://www.w3.org/ns/shacl#Warning,None,None,None,http://www.w3.org/ns/shacl#MinCountConstraintC...,https://schema.org/citation,Citation information should be provided,None,None,None


In [22]:
df.columns

Index(['id', 'name', 'severity', 'grname', 'str_refurl', 'str_refresrouce',
       'constraint', 'path', 'message', 'focusURL', 'focusType', 'value'],
      dtype='object')